## sqlite3のチュートリアル 

In [1]:
import sqlite3
from pathlib import Path

### データベースへの接続 

sqliteデータベースへ接続する．mySQLやMongoDBと違ってデータベース用のプロセスを立てておく必要は無いらしい．

In [2]:
db_path = Path("sample_db") / Path("my_sample.db")
conn = sqlite3.connect(db_path)
c = conn.cursor()

### テーブルの作成 

カーソルの`execute`メソッドを利用し，sqliteのcreate table文を利用する．テーブル名とカラムの名前と型を定義する．
以下では，`users`というテーブルを作成し，そのカラムを`id(integer), name(text)`としている．ここでPRIMARY KEYとは重複を許さないこと

In [5]:
c.execute("create table users (id integer PRIMARY KEY, name text NOT NULL)")

### データのインサート 

#### 文字列で行う場合 

In [6]:
c.execute("insert into users(id,name) values(1, 'John')")
conn.commit()

valuesの値がテーブルのカラムの数・型と一致する場合，カラム名を省略できる．

In [7]:
c.execute("insert into users values(2, 'John2')")
conn.commit()

カラムのうち幾つかを与える場合は，カラム名をしていする．後で確認するが，自動でidが割り振られている

In [8]:
c.execute("insert into users(name) values('John3')")
conn.commit()

#### プレースホルダ―で行う場合 

pythonオブジェクトを渡すことができる．リスト・タプルにしなければいけないことに注意

In [9]:
c.execute("insert into users values(?,?)",(4, "Steve"))
conn.commit()

#### リストのインサート 

In [10]:
multi_data = [(5, "Sato"),
              (6, "Kato")]
c.executemany("insert into users values(?, ?)",multi_data)
conn.commit()

### データの検索

#### すべてのデータの取得 

アスタリスクは全てのカラムを意味する．

In [11]:
c.execute("select * from users")
result = c.fetchall()
print(result)

[(1, 'John'), (2, 'John2'), (3, 'John3'), (4, 'Steve'), (5, 'Sato'), (6, 'Kato')]


In [12]:
c.execute("select name from users")
result = c.fetchall()
print(result)

[('John',), ('John2',), ('John3',), ('Steve',), ('Sato',), ('Kato',)]


In [13]:
c.execute("select id from users")
result = c.fetchall()
print(result)

[(1,), (2,), (3,), (4,), (5,), (6,)]


### 条件を利用したデータの検索

パターンマッチングなどの複雑な処理をしない条件検索の場合，where句を用いる

In [14]:
c.execute("select * from users where name = 'Steve'")
result = c.fetchall()
print(result)

[(4, 'Steve')]


In [15]:
c.execute("select * from users where id > 3")
result = c.fetchall()
print(result)

[(4, 'Steve'), (5, 'Sato'), (6, 'Kato')]


### 検索データのソート 

order by句で並べたいカラムの名前を指定する．また昇順の場合はASC, 降順の場合はDESCと指定する．

In [16]:
c.execute("select * from users where id > 3 order by id desc")
result = c.fetchall()
print(result)

[(6, 'Kato'), (5, 'Sato'), (4, 'Steve')]


### データの更新 

update テーブル名 setとして記述する．これらの句はプレースホルダ―も利用できる．

In [17]:
c.execute("update users set name = ? where id = ?", ('Tanaka', 5))
conn.commit()

c.execute("select * from users")
result = c.fetchall()
print(result)

[(1, 'John'), (2, 'John2'), (3, 'John3'), (4, 'Steve'), (5, 'Tanaka'), (6, 'Kato')]


もし存在しない条件をupdateしたらinsertされない.これでは，executemanyは使えず，さがしてif文で分岐する必要がでてきてしまう．

In [18]:
c.execute("update users set name = ? where id = ?", ('Ueda', 7))
conn.commit()

c.execute("select * from users")
result = c.fetchall()
print(result)

[(1, 'John'), (2, 'John2'), (3, 'John3'), (4, 'Steve'), (5, 'Tanaka'), (6, 'Kato')]


よってupsertは以下のように行う.[参考](https://qastack.jp/programming/15277373/sqlite-upsert-update-or-insert)

In [19]:
sql = """
insert into users (id, name)
  values(?, ?)
  on conflict(id)
  do update set name=excluded.name
"""
c.execute(sql,[7,'Ueda'])
conn.commit()

c.execute("select * from users")
result = c.fetchall()
print(result)

[(1, 'John'), (2, 'John2'), (3, 'John3'), (4, 'Steve'), (5, 'Tanaka'), (6, 'Kato'), (7, 'Ueda')]


### データの削除 

delete句を利用する．プレースホルダーはリスト・タプルでなければいけないことに注意．

In [20]:
c.execute("delete from users where id = ?",(3,))
conn.commit()

c.execute("select * from users")
result = c.fetchall()
print(result)

[(1, 'John'), (2, 'John2'), (4, 'Steve'), (5, 'Tanaka'), (6, 'Kato'), (7, 'Ueda')]


### カラムの追加 

alter table テーブル名 add column カラム名　データ型　とする．もちろんNullデータになるので使えない制約もある．

In [21]:
c.execute("alter table users add column year integer")

In [22]:
c.execute("update users set year = 17 where id = 5")
conn.commit()

c.execute("select * from users")
result = c.fetchall()
print(result)

[(1, 'John', None), (2, 'John2', None), (4, 'Steve', None), (5, 'Tanaka', 17), (6, 'Kato', None), (7, 'Ueda', None)]


### カラム構造の取得

おそらく二つ目がカラム名で，三つ目が型．詳しくは[こちら](https://ez-net.jp/article/57/MdNBzKNC/b7Qzbu0oeix0/)

In [23]:
c.execute("pragma table_info('users')")
col_list = c.fetchall()
print(col_list)

[(0, 'id', 'integer', 0, None, 1), (1, 'name', 'text', 1, None, 0), (2, 'year', 'integer', 0, None, 0)]


### テーブル構造の取得 

二つ目がtableのnameで三つ目がtablenameらしい．違いは?

In [25]:
c.execute("create table users2 (id integer)")
c.execute("select * from sqlite_master")
table_list = c.fetchall()
print(table_list)

[('table', 'users', 'users', 2, 'CREATE TABLE users (id integer PRIMARY KEY, name text NOT NULL, year integer)'), ('table', 'users2', 'users2', 3, 'CREATE TABLE users2 (id integer)')]


### テーブルの消去 

In [4]:
c.execute("drop table users")

### データベースのクローズ 

In [98]:
conn.close()